# Model Strangelog
Statistical approach to label anomalies in event logs based on percentile analysis of tokenized events.

It returns a labelled dataframe from a test df_trace with new column 'strangeness' for each event:

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

**Requires**
- log dataset from high level task
- LogColor instance pretrained with similar dataset
- A subset column with a partition of similar task executions. For example, TPL_ID.

### Example: traces grouped by TPL_ID

In [1]:
# Init the public dataset
from eliana.datasets import ParlogsObservations
logs = ParlogsObservations(system="GRAVITY", period="1w")

In [2]:
# Partition in success and errors
meta_success = logs.meta[ logs.meta['ERROR'] == False ]
meta_errors  = logs.meta[ logs.meta['ERROR'] == True ]
print(f'In {logs.system}, from {len(logs.meta)} executions, Success: {len(meta_success)}; Error: {len(meta_errors)}')

In GRAVITY, from 152 executions, Success: 128; Error: 24


In [3]:
# Train Log Colorizer
traces = logs.traces()
print(f'Loaded {len(traces)} traces')

Loaded 316207 traces


In [4]:
from eliana.preprocessing import LogColorizer, VltTokenizer
colorizer = LogColorizer(tokenizer=VltTokenizer(to_lowercase=False))
_ = colorizer.fit(traces, warn=True)

## Train Strangelog model

In [5]:
from eliana.models import Strangelog
subset_col='TPL_ID'

model = Strangelog(tokenizer=colorizer, subset_col=subset_col)
model.fit(meta_success, traces, warn=True)

## Predict for SUCCESS executions

SUCCESS should have very low instances of rare (2) and unexpected (3) events

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

In [6]:
for row in meta_success.sample(5).itertuples():
    print(f"Strangeness for {row.TPL_ID} (Error = {row.ERROR})")
    labelled = model.predict(row.TPL_ID, traces[ traces['trace_id'] == row.Index ] )
    display( labelled.groupby('strangeness').size().reset_index(name='count'))


Strangeness for GRAVITY_gen_tec_checkFddl (Error = False)


,strangeness,count
0,0,327
1,1,1


Strangeness for GRAVITY_gen_tec_AcqFibAOnAxis (Error = False)


,strangeness,count
0,0,672
1,1,23


Strangeness for P2VMWAVESC (Error = False)


,strangeness,count
0,0,3908
1,1,250
2,2,3


Strangeness for GRAVITY_gen_tec_SaveDb (Error = False)


,strangeness,count
0,0,121


Strangeness for P2VM (Error = False)


,strangeness,count
0,0,5456


## Predict for ERROR executions

ERRORS should have hign number of rare (2) and unexpected (3) events. And in general, the content of the unexpected should notify about the cause of errors.

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

In [8]:
for row in meta_errors.sample(5).itertuples():
    print(f"-------\nUnexpected events in {row.TPL_ID} (Error = {row.ERROR})")
    trace = traces[ traces['trace_id'] == row.Index ]
    labelled = model.predict(row.TPL_ID, trace )
    unexpected = labelled[ labelled['strangeness'] == 3 ]
    unexpected['color'] = unexpected['event'].apply(colorizer.tokenize)
    print()
    print( unexpected['color'].unique()[:20] )
    display( labelled.groupby('strangeness').size().reset_index(name='count'))


-------
Unexpected events in GRAVITY_gen_cal_init (Error = True)

['FLOG logManager UNFORSEEN template error'
 'ERR gvimetServer tacERR_MOD_BLOCK Unable to modify block parameters controlMet'
 'ERR gvimetServer tacERR_BLOCK_NAME Unknown block controlMet'
 'ERR gvimetServer tacERR_SEND_REQUEST Request to RTController failed tacSHMSG_MODIFY_BLOCK'
 'ERR bob_{} modify block parameters controlMet red'
 'ERR bob_{} block parameters controlMet red']


,strangeness,count
0,0,131
1,1,98
2,2,2
3,3,6


-------
Unexpected events in GRAVITY_single_acq (Error = True)

['FLOG logManager UNFORSEEN template error'
 'ERR bob_{} Error during setup states red']


,strangeness,count
0,0,1277
1,1,284
2,2,4
3,3,3


-------
Unexpected events in GRAVITY_dual_acq (Error = True)

['LOG bob_{} SEQ FT MODE {}' 'LOG bob_{} SEQ FT ROBJ NAME AlphaCenB'
 'LOG bob_{} SEQ INS SOBJ NAME {}' 'LOG bob_{} SEQ {} {}'
 'LOG logBackupUtil Extract FITS logs from day {} to {}'
 'LOG logBackupUtil Storing normal logs for the archive in file logFileArchive'
 'LOG logBackupUtil Directory DHS_DATA set to vltdata ARCDATA'
 'LOG logBackupUtil Append FITS logs to currentDay ops log'
 'LOG bob_{} Setting up SP mode IN HIGH brown'
 'LOG bob_{} Send command CONFMOD SP IN HIGH to process gvmonControl@wgv'
 'LOG gvspcControl gvspcMAIN_TASK C {} Init Deregistered socket {} gvspcControl from rtdcoreServer'
 'LOG gvspcControl gvspcMAIN_TASK C {} StopAllActions To be implemented'
 'LOG gvspcControl gvspcMAIN_TASK C {} Init Registered as gvspcControl at rtdcoreServer with socket {}'
 'LOG gvspcControl New subMode to be set {}'
 'LOG gvspcControl gvspcMAIN_TASK C {} SetMode pixel and calibration files read'
 'LOG bob_{} User set FT mo

,strangeness,count
0,0,7023
1,2,155
2,3,185


-------
Unexpected events in GRAVITY_gen_cal_init (Error = True)

['ERR gvimtfsMon000 gviserERR_COM_TIMEOUT Serial device timeout fd {}'
 'ERR gvimtfsMon000 gvimtfsERR_GENERAL Device mtfs Error reading from serial port'
 'ERR gvimtfsMon000 gvimtfsERR_GENERAL Device mtfs Sending Cmd receiving Reply for CON?'
 'FLOG logManager UNFORSEEN template error'
 'ERR gvimtfsServer gvimtfsERR_GENERAL Device mtfs Cannot take semaphore after'
 'ERR gvimtfsServer gvimtfsERR_CMD_REPLY Device mtfs Sending Cmd receiving Reply for CON {} {} failed'
 'ERR gvoControl bossERR_SUBSYSTEM_REPLY Error reply received from sub system ICS for command SETUP'
 'ERR gvimtfsServer lcctooERR_SEM_TAKE Take semaphore failed vxWorks error {}'
 'ERR gvoControl evhERR_CMD_ERR_REPLY Received an error reply to command SETUP from process gviControl on environment wgv'
 'ERR gvoControl bossERR_CMD_FAILED Error during the execution of command SETUP'
 'LOG gvoControl SETUP command failed'
 'ERR bob_{} Error during the execution o

,strangeness,count
0,0,53
1,1,86
2,2,3
3,3,16


-------
Unexpected events in GRAVITY_gen_tec_checkMetZero (Error = True)

['LOG gvmonControl errAdd parameter error is a NULL pointer errID {} module db location dbReadDirect c {}'
 'LOG gvmonControl errAdd parameter error is a NULL pointer errID {} module cai location caiGetAttributes c {}'
 'LOG gvmonControl errAdd parameter error is a NULL pointer errID {} module cai location caiRWAttribute c {}'
 'FLOG logManager UNFORSEEN {} LOCAL alias lgvmet scanningStatus {} Normal Scan system alarm for device pointName check device status'
 'FLOG logManager UNFORSEEN {} LOCAL alias lgvmet scanningStatus {} Fatal Scan system alarm for device pointName check device status'
 'LOG alrmLogger wgv {} LOCAL alias lgvmet scanningStatus {} Normal'
 'LOG alrmLogger wgv {} LOCAL alias lgvmet scanningStatus {} Fatal'
 'ERR ccsScanInit ccsERR_SCAN_FCT_CALL Error occurred in function ccsScanSend'
 'ERR bob_{} ccsERR_DB_QUALITY Point alias LCU lgvmet tacSimulation has Bad Data Quality SUSPECT'
 'ERR bob_{} s

,strangeness,count
0,0,1759
1,2,12
2,3,4031
